# 02 RAG | 03 Vector DB | 01 Azure AI Search

## Step 1: Create Search Index Client




In [11]:
#r "nuget: Azure.Search.Documents, 11.6.0-beta.4"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using DotNetEnv; 
using System.IO;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";;
string searchApiKey = Environment.GetEnvironmentVariable("WS_SEARCH_APIKEY") ?? "WS_SEARCH_APIKEY not found";
string searchEndpoint = Environment.GetEnvironmentVariable("WS_SEARCH_ENDPOINT") ?? "WS_SEARCH_ENDPOINT not found";


AzureKeyCredential azureKeyCredential = new AzureKeyCredential(searchApiKey);
SearchIndexClient searchIndexClient = new SearchIndexClient(new Uri(searchEndpoint), azureKeyCredential);

Console.WriteLine($"SearchIndexClient create...");


Installed Packages Azure.Search.Documents, 11.6.0-beta.4 DotNetEnv, 2.5.0

SearchIndexClient create...


# Step 2: Create Search Index


In [8]:
string indexName = "documenttenant"; 
string vectorSearchProfileNameHNSW = "search-profile-HNSW";
string vectorSearchProfileNameExhaustive = "search-profile-exhaustive";
string vectorSearchConfigHNSW = "vector-config-hnsw";
string vectorSearchConfigExhaustive = "vector-config-exhaustive";

int modelDimensions = 1536;

SearchIndex index = new SearchIndex(indexName)
{
    Fields =
    {
        new SimpleField("DocumentId", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true },
        new SearchableField("DocumentName") { IsFilterable = true, IsSortable = true},
        new SearchableField("Description") { AnalyzerName = LexicalAnalyzerName.EnLucene },
        new SearchableField("DescriptionGer") { AnalyzerName = LexicalAnalyzerName.DeLucene },
        new SearchableField("SummarizedContent") { IsFilterable = true, IsSortable = true },

        new ComplexField("MetaInfo")
        {
            Fields =
            {
                new SearchableField("Author") { IsFilterable = true, IsSortable = true, IsFacetable = true},
                new SearchableField("CreationDate") { IsFilterable = true, IsSortable = true, IsFacetable = true },
                new SearchableField("AuthorCountry") { IsFilterable = true, IsSortable = true, IsFacetable = true },
            }
        },
        new VectorSearchField("DescriptionVectorHNSW", modelDimensions, vectorSearchProfileNameHNSW),
        new VectorSearchField("DescriptionVectorExhaust", modelDimensions, vectorSearchProfileNameExhaustive)
    },
    VectorSearch = new() {
        Profiles =
        {
            new VectorSearchProfile(vectorSearchProfileNameHNSW, vectorSearchConfigHNSW), 
            new VectorSearchProfile(vectorSearchProfileNameExhaustive, vectorSearchConfigExhaustive)
        },
        Algorithms = {
            new ExhaustiveKnnAlgorithmConfiguration(vectorSearchConfigExhaustive) {
                Parameters = new ExhaustiveKnnParameters {   
                    Metric = VectorSearchAlgorithmMetric.Cosine
                }
            },
            new HnswAlgorithmConfiguration(vectorSearchConfigHNSW){
                Parameters = new HnswParameters
                {
                    EfConstruction = 200,
                    EfSearch = 200,
                    M = 4,
                    Metric = VectorSearchAlgorithmMetric.Cosine
                }
            }
        }
    }
};

await searchIndexClient.CreateOrUpdateIndexAsync(index);

## Step 3: Define Data Structure

In [10]:
using System.Text.Json.Serialization;

public class KnowledgeDocument {

    public string DocumentId { get; set; } = Guid.NewGuid().ToString();
    public string DocumentName { get; set; } = "";
    public string Description { get; set; } = "";
    public string DescriptionGer { get; set; } = "";
    public string SummarizedContent { get; set; } = "";
    public MetaInfo MetaInfo { get; set; } = new MetaInfo();

    // [JsonIgnore] Because property needs to be ignored when serializing and deserializing. 
    // Azure AI Search index does not implement this property
    // Value will be filled after search query with search score
    [JsonIgnore]        
    public double? Score { get; set; } = 0.0f;

}

public class KnowledgeDocumentVector : KnowledgeDocument {
    public float[] DescriptionVectorHNSW { get; set; } = new float[1536];
    public float[] DescriptionVectorExhaust { get; set; } = new float[1536];
}


public class MetaInfo {
    public string Author { get; set; } = "";
    public string CreationDate { get; set; } = "";
    public string AuthorCountry { get; set; } = "";
}


List<KnowledgeDocumentVector> knowledgeDocumentsVector = new List<KnowledgeDocumentVector>(); 

knowledgeDocumentsVector.Add(new KnowledgeDocumentVector()
{
    DocumentName = "Contoso-Music",
    Description = "Contoso Music is one of the world's leading record labels. It has signed famous singers and is very profitable! The flagship of Contoso Music is a group that performs under the name 'Contoso Only'!",
    DescriptionGer = "Contoso Musis ist eines der weltweit führenden Plattenlabel. Es hat berühmte Sänger unter Vertrag genommen und ist sehr profitabel! Das Flaggschiff von Contoso Music ist eine Gruppe, die unter dem Namen 'Contoso Only' auftritt!",
    MetaInfo = new MetaInfo()
    {
        Author = "Music",
        CreationDate = "2021-10-01",
        AuthorCountry = "Germany"
    }
});
knowledgeDocumentsVector.Add(new KnowledgeDocumentVector()
{
    DocumentName = "Contoso-Maritim",
    Description = "Contoso Heavy Industry Maritime products. The current bestseller is the container transporter 'Contoso XL Heavy 2000'.",
    DescriptionGer = "Contoso Heavy Industry Maritime Produkte. Der aktuelle Bestseller ist der Containertransporter 'Contoso XL Heavy 2000'.",
    MetaInfo = new MetaInfo()
    {
        Author = "Maritim",
        CreationDate = "2021-10-01",
        AuthorCountry = "Germany"
    }
});
knowledgeDocumentsVector.Add(new KnowledgeDocumentVector()
{
    DocumentName = "Contoso-Agriculture",
    Description = "Contoso Agriculture is a German start-up that focuses on the production of milk and grain. Since this is a start-up, no further information is available!",
    DescriptionGer = "Contoso Agriculture ist ein deutsches Start-up, das sich auf die Produktion von Milch und Getreide konzentriert. Da es sich um ein Start-up handelt, sind keine weiteren Informationen verfügbar!",
    MetaInfo = new MetaInfo()
    {
        Author = "Agriculture",
        CreationDate = "2021-10-01",
        AuthorCountry = "Germany"
    }
});

foreach(KnowledgeDocumentVector knowledgeDocument in knowledgeDocumentsVector) {
    knowledgeDocument.DescriptionVectorHNSW = new float[1536];
    knowledgeDocument.DescriptionVectorExhaust = new float[1536];
}


Step 4: Index Documents


In [12]:
SearchClient searchClient = searchIndexClient.GetSearchClient(indexName);
Response<IndexDocumentsResult> response = await searchClient.UploadDocumentsAsync<KnowledgeDocumentVector>(knowledgeDocumentsVector);

if (response.GetRawResponse().Status == 200) {
    return true;
} else {
    return false;
}

True